In [ ]:
class Powerset(TypedExpr):
    canonical_name = "Powerset"
    op_name_uni = "ℙ"
    op_name_latex = "\\mathbb{P}"
    pre_simplify = True

    def __init__(self, arg, typ=None, **kwargs):
        self.type_constraint(arg.type, types.SetType, constant=True)
        setarg, typ = self.type_constraint(SetType(arg), typ)
        arg, _ = self.type_constraint(arg, setarg[0])
    
        super().__init__(var, body, restrictor=restrictor, typ=body.type, **kwargs)

    def eliminate(self, assignment=None, **sopts):
        if self.vacuous():
            # covers empty set / domain set as body
            return derived(self.body.copy(), self, f"trivial ∪{self.varname} elimination")
        elif self.finite_safe():
            a = self.scope_assignment(assignment=assignment)
            subs = [self[1].under_assignment(a | {self.varname : elem})
                    for elem in self.domain_iter()]
            return derived(sets.SetUnion.join(subs, empty=true_term),
                self,
                f"∪{self.varname} => ∪")
        # disable this for now -- it produces more complex formulas, not simpler ones
        # else:
        #     return self.to_exists_form()

        return self

    def simplify(self, assignment=None, **sopts):
        if (self.vacuous() or self.domain_cardinality() == 1
                    or get_sopt('eliminate_sets', sopts)):
            return self.eliminate(assignment=assignment, **sopts)
        return self